In [1]:
import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import random as rn
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import codecs
from tensorflow.contrib.rnn import *

Using TensorFlow backend.


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:
# load feature and labels
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')
# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

# standardization
scaled_feature = False

In [4]:
split = 7869

In [5]:
#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
data = np.load('../../dimensionalSM/IEMOCAP_full_release/data_collected_10039.pickle', allow_pickle=True)
text = [t['transcription'] for t in data]
#vad = np.load('../../data/y_egemaps.npy')
# load fastext model
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_train_text = []
x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

word_index = tokenizer.word_index
nb_words = len(word_index) +1
print('Found %s unique tokens' % len(word_index))

# load model
EMBEDDING_DIM = 300
file_loc = '../../dimensionalSM/data/glove.42B.300d.txt'
print (file_loc)

gembeddings_index = {}
with codecs.open(file_loc, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

nb_words = len(word_index) +1
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

# split train/test

earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)


Found 3437 unique tokens
../../dimensionalSM/data/glove.42B.300d.txt
G Word embeddings: 1917494
G Null word embeddings: 128


In [6]:
if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1, keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss


In [7]:
# API model
def model():
    # speech network
    input_speech = Input(shape=(feat.shape[1], feat.shape[2]), name='speech_input')
    net_speech = BatchNormalization()(input_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(128, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    model_speech = Flatten()(net_speech)
    #model_speech = Dropout(0.1)(net_speech)
    
    #text network
    input_text = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net_text = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(input_text)
    net = Bidirectional(LSTM(128, return_sequences=True))(net_text)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    model_text = Dense(64)(net)

    # combined model
    model_combined = concatenate([model_speech, model_text])
    model_combined = Dense(64, activation='relu')(model_combined)
    model_combined = Dense(32, activation='relu')(model_combined)
    #model_combined = Dropout(0.4)(model_combined)
    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_combined) for name in target_names]

    model = Model([input_speech, input_text], model_combined) 
    model.compile(loss=ccc_loss,
                  loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


# model_1 = model()
# model_1.summary()

In [8]:
# #speech_to_npy
# model1 = model()
# earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
#                               restore_best_weights=True)
# hist1 = model1.fit([feat[:split],x_train_text[:split]], vad[:split].T.tolist(), epochs=50, 
#                    batch_size=32, verbose=1, validation_split=0.2, 
#                    callbacks=[earlystop])

# eval_metrik1 = model1.evaluate([feat[split:],x_train_text[split:]], vad[split:].T.tolist())
# print(eval_metrik1)

# # make prediction
# predict = model1.predict([feat[6290:],x_train_text[6290:]], batch_size=32)
# np.save('speech_text_npy7869', 
#          np.array(predict).reshape(3, 3749).T)


In [9]:
# main function for for LOSO
def main(time):
    model_1 = model()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [feat[:split],x_train_text[:split]],
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [feat[split:],x_train_text[split:]], vad[split:].T.tolist())
    # make prediction
    predict = model_1.predict([feat[6290:],x_train_text[6290:]], batch_size=32)
    np.save('npy7869/speech_text_npy-7869glove'+str(time)+'.npy', 
             np.array(predict).reshape(3, 3749).T)
    print(metrik[-3:])
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('speech_textglove_7869.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 48s 8ms/step - loss: 1.5430 - v_loss: 0.6179 - a_loss: 0.4156 - d_loss: 0.5084 - v_ccc: 0.3815 - a_ccc: 0.5841 - d_ccc: 0.4914 - val_loss: 2.0375 - val_v_loss: 0.6744 - val_a_loss: 0.7185 - val_d_loss: 0.6495 - val_v_ccc: 0.3309 - val_a_ccc: 0.2856 - val_d_ccc: 0.3460
Epoch 2/50
6295/6295 [==============================] - 42s 7ms/step - loss: 1.1125 - v_loss: 0.3814 - a_loss: 0.3182 - d_loss: 0.4133 - v_ccc: 0.6189 - a_ccc: 0.6818 - d_ccc: 0.5869 - val_loss: 1.6390 - val_v_loss: 0.5850 - val_a_loss: 0.5263 - val_d_loss: 0.5347 - val_v_ccc: 0.4183 - val_a_ccc: 0.4780 - val_d_ccc: 0.4648
Epoch 3/50
6295/6295 [==============================] - 41s 7ms/step - loss: 0.9844 - v_loss: 0.3095 - a_loss: 0.2946 - d_loss: 0.3799 - v_ccc: 0.6903 - a_ccc: 0.7052 - d_ccc: 0.6201 - val_loss: 1.6042 - val_v_loss: 0.67

Epoch 9/50
6295/6295 [==============================] - 49s 8ms/step - loss: 0.5908 - v_loss: 0.1527 - a_loss: 0.2073 - d_loss: 0.2307 - v_ccc: 0.8472 - a_ccc: 0.7925 - d_ccc: 0.7694 - val_loss: 1.5649 - val_v_loss: 0.5340 - val_a_loss: 0.4800 - val_d_loss: 0.5632 - val_v_ccc: 0.4742 - val_a_ccc: 0.5240 - val_d_ccc: 0.4369
Epoch 10/50
6295/6295 [==============================] - 49s 8ms/step - loss: 0.5432 - v_loss: 0.1402 - a_loss: 0.1917 - d_loss: 0.2111 - v_ccc: 0.8597 - a_ccc: 0.8083 - d_ccc: 0.7888 - val_loss: 1.5706 - val_v_loss: 0.5328 - val_a_loss: 0.4819 - val_d_loss: 0.5745 - val_v_ccc: 0.4772 - val_a_ccc: 0.5239 - val_d_ccc: 0.4283
Epoch 11/50
6295/6295 [==============================] - 49s 8ms/step - loss: 0.4935 - v_loss: 0.1318 - a_loss: 0.1702 - d_loss: 0.1914 - v_ccc: 0.8681 - a_ccc: 0.8298 - d_ccc: 0.8086 - val_loss: 1.5612 - val_v_loss: 0.5354 - val_a_loss: 0.4925 - val_d_loss: 0.5487 - val_v_ccc: 0.4720 - val_a_ccc: 0.5134 - val_d_ccc: 0.4534
Epoch 12/50
6295/6295 [

6295/6295 [==============================] - 51s 8ms/step - loss: 1.1193 - v_loss: 0.3803 - a_loss: 0.3224 - d_loss: 0.4166 - v_ccc: 0.6196 - a_ccc: 0.6779 - d_ccc: 0.5833 - val_loss: 1.6786 - val_v_loss: 0.6539 - val_a_loss: 0.5053 - val_d_loss: 0.5283 - val_v_ccc: 0.3495 - val_a_ccc: 0.4996 - val_d_ccc: 0.4723
Epoch 3/50
6295/6295 [==============================] - 50s 8ms/step - loss: 0.9898 - v_loss: 0.3095 - a_loss: 0.3001 - d_loss: 0.3808 - v_ccc: 0.6904 - a_ccc: 0.7002 - d_ccc: 0.6197 - val_loss: 1.5829 - val_v_loss: 0.5828 - val_a_loss: 0.4718 - val_d_loss: 0.5458 - val_v_ccc: 0.4232 - val_a_ccc: 0.5346 - val_d_ccc: 0.4593
Epoch 4/50
6295/6295 [==============================] - 50s 8ms/step - loss: 0.8945 - v_loss: 0.2609 - a_loss: 0.2812 - d_loss: 0.3522 - v_ccc: 0.7391 - a_ccc: 0.7187 - d_ccc: 0.6477 - val_loss: 1.4977 - val_v_loss: 0.5918 - val_a_loss: 0.4197 - val_d_loss: 0.4969 - val_v_ccc: 0.4164 - val_a_ccc: 0.5850 - val_d_ccc: 0.5009
Epoch 5/50
6295/6295 [==============

Epoch 13/50
2170/2170 [==============================] - 5s 2ms/step
[0.4230157136917114, 0.5818215012550354, 0.5162786841392517]
4
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 58s 9ms/step - loss: 1.5444 - v_loss: 0.6096 - a_loss: 0.4110 - d_loss: 0.5236 - v_ccc: 0.3901 - a_ccc: 0.5889 - d_ccc: 0.4765 - val_loss: 2.0658 - val_v_loss: 0.6695 - val_a_loss: 0.7356 - val_d_loss: 0.6572 - val_v_ccc: 0.3286 - val_a_ccc: 0.2666 - val_d_ccc: 0.3389
Epoch 2/50
6295/6295 [==============================] - 51s 8ms/step - loss: 1.1128 - v_loss: 0.3794 - a_loss: 0.3187 - d_loss: 0.4146 - v_ccc: 0.6205 - a_ccc: 0.6813 - d_ccc: 0.5854 - val_loss: 1.6462 - val_v_loss: 0.6579 - val_a_loss: 0.4776 - val_d_loss: 0.5200 - val_v_ccc: 0.3442 - val_a_ccc: 0.5294 - val_d_ccc: 0.4802
Epoch 3/50
6295/6295 [==============================] - 51s 8ms/step - loss: 0.9779 - v_loss: 0.3069 - a_loss: 0.2901 - d_loss: 0.3807 - v_ccc: 0.6930 - a_ccc: 0.7098 - d

6295/6295 [==============================] - 51s 8ms/step - loss: 0.5842 - v_loss: 0.1532 - a_loss: 0.1987 - d_loss: 0.2323 - v_ccc: 0.8467 - a_ccc: 0.8013 - d_ccc: 0.7679 - val_loss: 1.5314 - val_v_loss: 0.5414 - val_a_loss: 0.4594 - val_d_loss: 0.5625 - val_v_ccc: 0.4718 - val_a_ccc: 0.5474 - val_d_ccc: 0.4494
Epoch 10/50
6295/6295 [==============================] - 51s 8ms/step - loss: 0.5282 - v_loss: 0.1422 - a_loss: 0.1816 - d_loss: 0.2044 - v_ccc: 0.8579 - a_ccc: 0.8184 - d_ccc: 0.7955 - val_loss: 1.6010 - val_v_loss: 0.5869 - val_a_loss: 0.4681 - val_d_loss: 0.5691 - val_v_ccc: 0.4222 - val_a_ccc: 0.5365 - val_d_ccc: 0.4403
Epoch 11/50
6295/6295 [==============================] - 50s 8ms/step - loss: 0.4893 - v_loss: 0.1314 - a_loss: 0.1666 - d_loss: 0.1912 - v_ccc: 0.8685 - a_ccc: 0.8334 - d_ccc: 0.8088 - val_loss: 1.5812 - val_v_loss: 0.5392 - val_a_loss: 0.4870 - val_d_loss: 0.5857 - val_v_ccc: 0.4717 - val_a_ccc: 0.5207 - val_d_ccc: 0.4264
Epoch 12/50
6295/6295 [===========

...

2170/2170 [==============================] - 5s 2ms/step
[0.40097230672836304, 0.5729464888572693, 0.4904347062110901]
15
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 64s 10ms/step - loss: 1.5867 - v_loss: 0.6185 - a_loss: 0.4284 - d_loss: 0.5388 - v_ccc: 0.3809 - a_ccc: 0.5715 - d_ccc: 0.4609 - val_loss: 2.0739 - val_v_loss: 0.7424 - val_a_loss: 0.6746 - val_d_loss: 0.6743 - val_v_ccc: 0.2609 - val_a_ccc: 0.3323 - val_d_ccc: 0.3329
Epoch 2/50
6295/6295 [==============================] - 51s 8ms/step - loss: 1.1344 - v_loss: 0.3863 - a_loss: 0.3238 - d_loss: 0.4238 - v_ccc: 0.6137 - a_ccc: 0.6759 - d_ccc: 0.5760 - val_loss: 1.5856 - val_v_loss: 0.6162 - val_a_loss: 0.4741 - val_d_loss: 0.5098 - val_v_ccc: 0.3878 - val_a_ccc: 0.5332 - val_d_ccc: 0.4935
Epoch 3/50
6295/6295 [==============================] - 51s 8ms/step - loss: 0.9965 - v_loss: 0.3122 - a_loss: 0.2982 - d_loss: 0.3860 - v_ccc: 0.6877 - a_ccc: 0.7020 - d_ccc: 0.6

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6295/6295 [==============================] - 51s 8ms/step - loss: 0.3649 - v_loss: 0.0993 - a_loss: 0.1281 - d_loss: 0.1376 - v_ccc: 0.9007 - a_ccc: 0.8719 - d_ccc: 0.8625 - val_loss: 1.5522 - val_v_loss: 0.5372 - val_a_loss: 0.4856 - val_d_loss: 0.5583 - val_v_ccc: 0.4757 - val_a_ccc: 0.5225 - val_d_ccc: 0.4496
Epoch 16/50
2170/2170 [==============================] - 5s 2ms/step
[0.4208301901817322, 0.543430507183075, 0.4439067542552948]
17
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 65s 10ms/step - loss: 1.5630 - v_loss: 0.6062 - a_loss: 0.4281 - d_loss: 0.5293 - v_ccc: 0.3939 - a_ccc: 0.5723 - d_ccc: 0.4709 - val_loss: 2.2453 - val_v_loss: 0.7995 - val_a_loss: 0.7883 - val_d_loss: 0.6626 - val_v_ccc: 0.2044 - val_a_ccc: 0.2172 - val_d_ccc: 0.3330
Epoch 2/50
6295/6295 [==============================] - 52s 8ms/step - loss: 1.1229 - v_loss: 0.3935 - a_loss: 0.3161 - d_loss: 0.4133 - v_ccc: 0.6067 - a_ccc: 0.6839 - d_ccc: 0.58

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6295/6295 [==============================] - 51s 8ms/step - loss: 0.6855 - v_loss: 0.1831 - a_loss: 0.2332 - d_loss: 0.2691 - v_ccc: 0.8168 - a_ccc: 0.7669 - d_ccc: 0.7307 - val_loss: 1.5361 - val_v_loss: 0.5528 - val_a_loss: 0.4399 - val_d_loss: 0.5676 - val_v_ccc: 0.4581 - val_a_ccc: 0.5643 - val_d_ccc: 0.4415
Epoch 8/50
6295/6295 [==============================] - 51s 8ms/step - loss: 0.6206 - v_loss: 0.1654 - a_loss: 0.2130 - d_loss: 0.2419 - v_ccc: 0.8346 - a_ccc: 0.7869 - d_ccc: 0.7579 - val_loss: 1.5082 - val_v_loss: 0.5261 - val_a_loss: 0.4567 - val_d_loss: 0.5505 - val_v_ccc: 0.4845 - val_a_ccc: 0.5490 - val_d_ccc: 0.4583
Epoch 9/50
6295/6295 [==============================] - 51s 8ms/step - loss: 0.5775 - v_loss: 0.1536 - a_loss: 0.1999 - d_loss: 0.2242 - v_ccc: 0.8464 - a_ccc: 0.8002 - d_ccc: 0.7759 - val_loss: 1.5509 - val_v_loss: 0.5162 - val_a_loss: 0.4797 - val_d_loss: 0.5706 - val_v_ccc: 0.4897 - val_a_ccc: 0.5243 - val_d_ccc: 0.4351
Epoch 10/50
6295/6295 [=============